In [2]:
#MNISTにアンサンブル学習適応
def sort_by_target(mnist):
    reorder_train=np.array(sorted([(target, i)for i,target in enumerate(mnist.target[:60000])]))[:,1]
    reorder_test=np.array(sorted([(target,i)for i, target in enumerate(mnist.target[60000:])]))[:,1]
    mnist.data[:60000] = mnist.data[reorder_train]
    mnist.target[:60000]=mnist.target[reorder_train]
    mnist.data[60000:]=mnist.data[reorder_test+60000]
    mnist.target[60000:]=mnist.target[reorder_test+60000]

import numpy as np    
                                                                                            
from sklearn.datasets import fetch_openml
mnist=fetch_openml("mnist_784",version=1,cache=True)
mnist.target=mnist.target.astype(np.int8)
sort_by_target(mnist)
    
import numpy as np    
    
X=mnist["data"]
y=mnist["target"]

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=10000,random_state=42)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=10000,random_state=42)  ##mnist データサイズ指定

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier##ensenbleでまとめてimport
from sklearn.svm import LinearSVC

random_forest_clf=RandomForestClassifier(n_estimators=10,random_state=42) 
Extree_clf=ExtraTreesClassifier(n_estimators=10,random_state=42)
svm_clf=LinearSVC(random_state=42,dual = True)


estimators=[random_forest_clf,Extree_clf,svm_clf]
for estimator in estimators:   
    print("training the", estimator)
    estimator.fit(X_train,y_train)

print([estimator.score(X_val,y_val) for estimator in estimators])   

#投票分類
from sklearn.ensemble import VotingClassifier
named_estimators=[("random_forest",random_forest_clf),
                 ("Extree_clf",Extree_clf),
                 ("svm_clf",svm_clf)]

voting_clf=VotingClassifier(named_estimators)
voting_clf.fit(X_train,y_train)

print(voting_clf.score(X_val,y_val))

##投票アンサンブルの中の個別の分類器と比較する
print([estimator.score(X_val,y_val) for estimator in voting_clf.estimators_])

##ここでset_parms でSVMを除いてもトレーニングセットからは消えないの
del voting_clf.estimators_[2]

print(voting_clf.score(X_val,y_val))

voting_clf.voting="soft"   ###属性指定
print(voting_clf.score(X_val,y_val))

print(voting_clf.score(X_test,y_test))

[estimator.score(X_test,y_test) for estimator in voting_clf.estimators_]
   
      



training the RandomForestClassifier(n_estimators=10, random_state=42)
training the ExtraTreesClassifier(n_estimators=10, random_state=42)
training the LinearSVC(random_state=42)


C:\Users\a.sakata\anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[0.9467, 0.9512, 0.8051]


C:\Users\a.sakata\anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.9492
[0.9467, 0.9512, 0.8051]
0.9442
0.9625
0.9582


[0.9434, 0.9444]

In [1]:
#スタッキングアンサンブル
def sort_by_target(mnist):
    reorder_train=np.array(sorted([(target, i)for i,target in enumerate(mnist.target[:60000])]))[:,1]
    reorder_test=np.array(sorted([(target,i)for i, target in enumerate(mnist.target[60000:])]))[:,1]
    mnist.data[:60000] = mnist.data[reorder_train]
    mnist.target[:60000]=mnist.target[reorder_train]
    mnist.data[60000:]=mnist.data[reorder_test+60000]
    mnist.target[60000:]=mnist.target[reorder_test+60000]

import numpy as np    
                                                                                            
from sklearn.datasets import fetch_openml
mnist=fetch_openml("mnist_784",version=1,cache=True)
mnist.target=mnist.target.astype(np.int8)
sort_by_target(mnist)
    
import numpy as np    
    
X=mnist["data"]
y=mnist["target"]

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=10000,random_state=42)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=10000,random_state=42) 

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier##ensenbleでまとめてimport
from sklearn.svm import LinearSVC

random_forest_clf=RandomForestClassifier(n_estimators=10,random_state=42) 
Extree_clf=ExtraTreesClassifier(n_estimators=10,random_state=42)
svm_clf=LinearSVC(random_state=42)


estimators=[random_forest_clf,Extree_clf,svm_clf]
for estimator in estimators:   
    print("training the", estimator)
    estimator.fit(X_train,y_train)

X_val_predictions=np.empty((len(X_val),len(estimators)),dtype=np.float32)

for index,estimator in enumerate(estimators):
    X_val_predictions[:, index]=estimator.predict(X_val)
    
print(X_val_predictions)

rnd_forest_blender=RandomForestClassifier(n_estimators=200,
                                         oob_score=True,random_state=42)

rnd_forest_blender.fit(X_val_predictions,y_val)

print(rnd_forest_blender.oob_score_)

X_test_predictions=np.empty((len(X_test),len(estimators)),dtype=np.float32)

for index,estimator in enumerate(estimators):
    X_test_predictions[:,index]=estimator.predict(X_test)
    
    
y_pred=rnd_forest_blender.predict(X_test_predictions)
print(y_pred)

from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)




   
      


training the RandomForestClassifier(n_estimators=10, random_state=42)
training the ExtraTreesClassifier(n_estimators=10, random_state=42)
training the LinearSVC(random_state=42)


C:\Users\a.sakata\anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[[2. 2. 2.]
 [7. 7. 7.]
 [4. 4. 4.]
 ...
 [4. 4. 4.]
 [9. 9. 9.]
 [4. 4. 4.]]
0.9508
[7 8 6 ... 9 1 6]


0.9457